# Topic Modeling

-Unsupervised technique that helps analyze large volumes of text data by clustering texts into group
<br>
-topic modeling tries to group the documents into clusters based on similar characteristics
<br>
-difficult to evaluate the performance of topic modeling since there are no right answers


In [51]:
#import all packages and libraries

import pandas as pd
import re
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [52]:
#Import the file with reviews

data = pd.read_csv ("/Users/aniljoshi/Downloads/Reviews.csv")
data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [53]:
#selecting only the reviews for now to analyze.

reviews = data.loc[:,'reviews.text']
reviews[0:5]

0    I order 3 of them and one of the item is bad q...
1    Bulk is always the less expensive way to go fo...
2    Well they are not Duracell but for the price i...
3    Seem to work as well as name brand batteries a...
4    These batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [55]:
# Remove punctuations from the reviews
reviews = reviews.map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the reviews into lowercase
reviews = reviews.map(lambda x: x.lower())

#print reviews
reviews


0        i order 3 of them and one of the item is bad q...
1        bulk is always the less expensive way to go fo...
2        well they are not duracell but for the price i...
3        seem to work as well as name brand batteries a...
4        these batteries are very long lasting the pric...
                               ...                        
28327    i got 2 of these for my 8 yr old twins my 11 y...
28328    i bought this for my niece for a christmas gif...
28329    very nice for light internet browsing keeping ...
28330    this tablet does absolutely everything i want ...
28331    at ninety dollars the expectionations are low ...
Name: reviews.text, Length: 28332, dtype: object

In [56]:
# using sklearn, pulling only the words that appear in less than 80% of the reviews and in at least two reviews

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(reviews.values.astype('U'))

In [57]:
#Viewing the matrix. There are 7618 words in the vocab to use for in topics

doc_term_matrix

<28332x7618 sparse matrix of type '<class 'numpy.int64'>'
	with 302342 stored elements in Compressed Sparse Row format>

In [ ]:
#create topics along the probability distribution of the words appearing. Setting 5 topics for first test.

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

In [59]:
# Get feature names

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

winner
blinked
protectors
questionable
evenly
remote
sale
owner
tablet
bummer


In [ ]:
#Get the components in the first topic

first_topic = LDA.components_[0]

In [ ]:
# Get the top words and sort

top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([1353, 6248, 4113, 3122, 2986, 6627, 7568, 4678, 1029, 4118])

In [ ]:
# Get the top words in topic

for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

christmas
son
love
great
gift
tablet
year
old
bought
loves


In [ ]:
# Get top 10 words in all five topics

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['christmas', 'son', 'love', 'great', 'gift', 'tablet', 'year', 'old', 'bought', 'loves']


Top 10 words for topic #1:
['just', 'new', 'battery', 'amazon', 'light', 'better', 'device', 'like', 'screen', 'kindle']


Top 10 words for topic #2:
['size', 'amazon', 'books', 'good', 'price', 'love', 'easy', 'use', 'tablet', 'great']


Top 10 words for topic #3:
['games', 'bought', 'play', 'time', 'good', 'great', 'amazon', 'apps', 'kids', 'tablet']


Top 10 words for topic #4:
['brand', 'amazon', 'battery', 'just', 'work', 'long', 'price', 'good', 'great', 'batteries']


